# Claude Flow Quickstart

This notebook verifies your environment is set up correctly.

## 1. Environment Check

Run this cell to verify all paths are configured:

In [ ]:
from _paths import *
print_env()

## 2. ChromaDB Connection

Test connection to the knowledge base:

In [ ]:
client = get_chromadb_client()
collections = client.list_collections()
print(f"Connected to ChromaDB at {CHROMADB_PATH}")
print(f"Collections: {[c.name for c in collections]}")

if collections:
    kb = client.get_collection("knowledge_base")
    print(f"\nKnowledge base has {kb.count()} documents")

## 3. Knowledge Pipeline

Check discovered schemas from the ingestion pipeline:

In [ ]:
schemas = load_discovered_schemas()
fields = load_discovered_fields()

print(f"Discovered schemas: {len(schemas)} events")
print(f"Discovered fields:  {len(fields)} field paths")

if schemas:
    print("\nEvent types found:")
    for event_name in list(schemas.keys())[:10]:
        print(f"  - {event_name}")
    if len(schemas) > 10:
        print(f"  ... and {len(schemas) - 10} more")

## 4. RAG Query Test

Test semantic search over the knowledge base:

In [ ]:
try:
    from retrieval.retrieve import search
    
    results = search("gameStateUpdate price fields", top_k=3)
    print("Search results for 'gameStateUpdate price fields':")
    print("=" * 50)
    for i, (doc, meta, dist) in enumerate(zip(results['documents'][0], 
                                               results['metadatas'][0],
                                               results['distances'][0])):
        print(f"\n[{i+1}] Distance: {dist:.3f}")
        print(f"    Source: {meta.get('source', 'unknown')}")
        print(f"    Content: {doc[:200]}...")
except ImportError as e:
    print(f"RAG pipeline not available: {e}")
    print("Run: cd rag-pipeline && python -m ingestion.ingest")

## 5. Optional: RL Bot Connection

If rugs-rl-bot is configured, test the connection:

In [ ]:
if RUGS_RL_BOT_PATH and RUGS_RL_BOT_PATH.exists():
    import sys
    sys.path.insert(0, str(RUGS_RL_BOT_PATH))
    
    try:
        from rugs_bot.sidebet.predictor import SidebetPredictor
        print(f"RL Bot connected at: {RUGS_RL_BOT_PATH}")
        print("SidebetPredictor available")
    except ImportError as e:
        print(f"RL Bot found but import failed: {e}")
else:
    print("RL Bot not configured (optional)")
    print("Set RUGS_RL_BOT_PATH in jupyter/config.env to enable")

## Summary

If all cells above ran without errors, your environment is ready!

**Next steps:**
- `01_event_explorer.ipynb` - Browse discovered events
- `02_canonical_review.ipynb` - Review and approve field promotions
- `03_coverage_dashboard.ipynb` - See documentation coverage
- `04_rl_bot_analysis.ipynb` - Analyze RL models (if configured)